#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?

In [44]:
## The marketing airline is the airline that sold the ticket, and the operating carrier is the carrier that actually carried the passenger

# https://en.wikipedia.org/wiki/Codeshare_agreement

In [2]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
pd.set_option('display.max_rows', None)

In [4]:
df_flights = pd.read_csv(r'..\..\2_data\flights_random_sample.csv', sep=',')
df_flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-01-01,WN,WN,WN,5431,WN,N291WN,5431,12889,LAS,...,986,1.0,0.0,0.0,0.0,18.0,NaN,NaN,NaN,NaN
1,2018-01-01,WN,WN,WN,989,WN,N287WN,989,12889,LAS,...,1099,0.0,0.0,11.0,0.0,35.0,NaN,NaN,NaN,NaN
2,2018-01-01,WN,WN,WN,1664,WN,N751SW,1664,12889,LAS,...,1099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,WN,WN,WN,1106,WN,N704SW,1106,12889,LAS,...,197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,WN,WN,WN,1559,WN,N423WN,1559,12889,LAS,...,197,37.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [5]:
df_p = pd.read_csv(r'..\..\2_data\passenger_random_sample.csv', sep=',')
df_p.head()

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0,30,4000713,7514,6870,1341006,0,5487,0,0,...,JP,Japan,6,637,1,2019,6,11,F,IF
1,0,60,8075192,15490,13653,1791108,0,5451,0,0,...,JP,Japan,6,637,1,2019,6,11,F,IF
2,0,13,1922307,6575,4249,261279,0,3819,0,0,...,US,United States,8,882,1,2019,6,8,F,IF
3,0,47,4185601,11562,10753,1300134,0,3819,0,0,...,US,United States,6,889,1,2019,6,8,F,IF
4,0,30,4002948,7528,6237,1538975,0,6752,0,0,...,US,United States,6,637,1,2019,6,14,F,IF


In [8]:
df_fuel = pd.read_csv(r'..\..\2_data\fuel_consumption.csv', sep=',')
df_fuel.head()

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,NaN,0JQ,NaN,1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,5100,0.0,5100,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,396216,140239.0,536455,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,0,0.0,0,2016
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,0,0.0,0,2016
4,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,260848.0,0.0,0.0,0.0,...,522405,0.0,0.0,0.0,0.0,522405,569497,0.0,569497,2016


Step 1 - Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)


In [28]:
# create month column

df_flights['month'] = pd.DatetimeIndex(df_flights['fl_date']).month  

In [29]:
# get delay per month per carrier

filter = df_flights[['month','op_unique_carrier','dep_delay']] 
carrier_delay_monthly = filter.groupby(['op_unique_carrier','month']).sum('dep_delay')
carrier_delay_monthly.head(20)

dep_delay
op_unique_carrier month           
9E                1         1966.0
                  2         3446.0
                  3         1548.0
                  4         2389.0
                  5         1418.0
                  6         6088.0
                  7         6704.0
                  8         6625.0
                  9         1278.0
                  10        1792.0
                  11         351.0
                  12        5700.0
9K                3          -75.0
                  5         -100.0
AA                1         7399.0
                  2        10614.0
                  3        13192.0
                  4        10293.0
                  5        12256.0
                  6        16746.0

In [37]:
# get average monthly delay per carrier

avg_monthly_delay = carrier_delay_monthly.groupby('op_unique_carrier').mean('dep_delay').reset_index()
avg_monthly_delay

,op_unique_carrier,distance
0,9E,1.404173e+05
1,9K,1.354500e+03
2,AA,1.067937e+06
3,AS,2.917309e+05
4,AX,3.358417e+04
5,B6,3.937174e+05
6,C5,1.890067e+04
7,CP,5.044058e+04
8,DL,9.617262e+05
9,EM,1.716000e+03


Step 2 - Find out distance covered monthly by different air carriers

In [32]:
# get distance per month per carrier

filter = df_flights[['month','op_unique_carrier','distance']] 
carrier_delay_monthly = filter.groupby(['op_unique_carrier','month']).sum('distance')
carrier_delay_monthly.head(20)

distance
op_unique_carrier month          
9E                1        158454
                  2        120542
                  3        130213
                  4        116440
                  5         99519
                  6        149324
                  7        110829
                  8        210361
                  9        180432
                  10       139898
                  11       109724
                  12       159272
9K                3          1290
                  5          1419
AA                1       1111331
                  2        982814
                  3       1203523
                  4       1075037
                  5        856212
                  6       1152391

In [36]:
# get average monthly distance per carrier

avg_monthly_distance = carrier_delay_monthly.groupby('op_unique_carrier').mean('distance').reset_index()
avg_monthly_distance

,op_unique_carrier,distance
0,9E,1.404173e+05
1,9K,1.354500e+03
2,AA,1.067937e+06
3,AS,2.917309e+05
4,AX,3.358417e+04
5,B6,3.937174e+05
6,C5,1.890067e+04
7,CP,5.044058e+04
8,DL,9.617262e+05
9,EM,1.716000e+03


In [47]:
fuc = df_flights['op_unique_carrier'].unique()
len(fuc)

27

In [62]:
muc = df_flights['mkt_unique_carrier'].unique()
len(muc)

10

In [63]:
codeshare = df_flights['branded_code_share'].unique()
len(codeshare)

15

In [60]:
puc = df_p['unique_carrier'].unique()
len(puc)

329

Step 3 - Find out number of passengers that were carried by different air carriers

DL     10667
UA      8856
WN      8335
AA      7065
OO      6405
FX      5105
EV      3432
H6      3240
G4      2788
YX      2681
AS      2600
5X      2524
NK      2232
MQ      2227
F9      2187
OH      2078
B6      2001
9E      1950
GV      1863
YV      1486
8E      1372
SY      1299
7S      1230
G7      1005
09Q      835
ZW       797
5Y       751
AX       678
04Q      659
GL       638
27Q      616
CP       543
U7       504
8V       502
PT       502
Y4       492
KE       490
C5       470
S5       467
AC       452
WS       432
5V       427
K2       420
J5       417
EM       410
V8       390
LH       385
2O       371
QX       371
ABX      362
AMQ      357
13Q      334
0WQ      321
LF       318
US       313
4Y       306
KAQ      271
HA       268
BA       265
M6       260
DY       255
KAH      247
4O       246
XP       241
9K       236
AM       229
KO       223
7H       222
NC       222
QK       212
3F       208
K3       206
4W       206
GFQ      206
3M       203
VX       190
CA       180

In [41]:
# get passengers per month per carrier

filter = df_p[['month','unique_carrier','passengers']]
carrier_passengers_monthly = filter.groupby(['unique_carrier','month']).sum('passengers')
carrier_passengers_monthly.head(20)

passengers
unique_carrier month            
04Q            1             308
               2             255
               3             617
               4             323
               5             438
               6            2301
               7            3118
               8            1899
               9            2149
               10            260
               11           1356
               12            517
07Q            2              30
               3               0
09Q            1            8677
               2           14349
               3           21437
               4            6064
               5           12880
               6            2610